# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

    
<b>Описание данных</b>
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита.

## Обзор данных

Обзор данных начнём с чтения файла и вывода общей информации о получившемся датафрейме, чтобы выявить возможные ошибки в названиях столбцов и получить представление о наличии пропусков:

In [21]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


С названиями столбцов порядок, но есть **пропуски** в столбцах **total_income** и **days_employed**.
Выведем первые 10 строк таблицы, чтобы поближе ознакомиться с данными:

In [22]:
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В глаза бросаются аномалии, с которыми предстоит разобаться:
- столбец days_employed: отрицательные и нереально большое значение соответствующее порядку 1000 лет. 
- столбец education: значения записаны разным регистром

Пропусков не замечено в первых строках таблицы, поэтому поищем их:

In [23]:
display(data[data['days_employed'].isna() == True].head(10)) #сначала поищем в столбце 'days_employed'

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Обнаружены пропуски типа NaN, причём видно что, в строке с пропуском days_employed также присутствует пропуск в total_income. Из общей информации о датафрейме видно, что количество пропусков в двух столбцах одинаковое, поэтому считаю, что они встречаются в одних и тех же строках и причина их появления одна и та же.

**По моему мнению, подобное возможно, если пользователь при вводе данных не указал трудовой стаж или ежемесячный доход. В таком случае обе строки заполняются NaN. Или это ошибка выгрузки**

В строках с пропусками встречаются разные значения по остальным столбцам, поэтому считаю, что появление пропусков с ними не связано.

Нужно принять решение об удалении строк с пропусками или их заполнении. Для этого посчитаем долю пропусков в обоих столбцах.

In [24]:
entries = data['dob_years'].count()
entries_nan = entries - data['days_employed'].count()
entries_nan / entries

0.10099883855981417

10% пропусков это много. Их лучше заполнить медианным значением - оно не окажет критического влияния на правдивость исследования, т.к. в отличие от среднего значения, медиана не подвержена влиянию аномально больших/маленьких значений.

Кроме того, уже видно, что в столбцах с пропусками есть некорректные значения, поэтому считаю, что лучше сначала разобраться с ними, а потом заполнять пропуски

## Предобработка данных

В ходе размышлений над алгоритмом предобработки было принято решение сначала обрабатывать по очереди столбцы по всем возможным проблемам и артефактам: 

- пропуски
- неявные дубликаты
- аномальные значения
- некорректный/неудобный тип данных

Таким образом мы подготовим датафрейм к быстрому удалению дублирующих друг друга данных (ведь, к примеру, разный регистр может помешать распознать дубликат). Для этого понадобится всего одна строка кода.

### Работа со столбцами days_employed и total_income

При первом взгляде на данные таблицы мы уже увидели некоторые аномалии, которые было бы полезно исправить для дальнейшего исследования. Это отрицательные значения и аномально большие значения трудового стажа. Думаю, что оптимально будет сразу перевести их в целые положительные числа для удобства восприятия. А заодно переведем в int значения столбца total_income	

In [25]:
columns = ['days_employed', 'total_income']
for column in columns:
    data[column] = data[column].fillna(0) #заполним пропуски нулём для избежания ошибок при переводе значений столбцов в int
    data[column] = abs(data[column].astype('int')) #функция abs превратит минус в плюс :-)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


Как видим, что код сработал и данные в таблице читаются лучше.

Поищем аномально большие значения трудового стажа. "Границу аномальности" выберем равную 60 годам и посчитаем количество аномальных значений

In [26]:
data[data['days_employed'] > 60*365]['days_employed'].count()

3445

Большое количество подобных значений может быть связано с ошибочно написанной функцией, переводящей годы и месяцы трудового стажа из лет и месяцев в дни.

Заменим аномально большие значения медианным значением по этому столбцу и проверим пезультат

In [27]:
days_employed_median = int(data['days_employed'].median())
data.loc[(data['days_employed'] > 60*365), 'days_employed'] = days_employed_median
data[data['days_employed'] > 60*365]['days_employed'].count()

0

Аномально больших значений в столбце days_employed больше нет.
Теперь заменим нулевые значения на медиану и проверим результат

In [28]:
data.loc[(data['days_employed'] == 0), 'days_employed'] = days_employed_median
data[data['days_employed'] == 0]['days_employed'].count()

0

Вот и нулевых значений больше нет.
Переходим к замене нулевых значений в столбце total_income

In [29]:
total_income_median = int(data['total_income'].median())
data.loc[(data['total_income'] == 0), 'total_income'] = total_income_median
data[data['total_income'] == 0]['total_income'].count()

0

Нулевых значений в столбце total_income больше нет

### Работа со столбцом children

Выявим аномальные значения в столбце children. С помощью метода value_counts(), т.к. не ожидается большое количество уникальных значений, а информация по частоте появления даст понят, насколько данные пригодны для анализа.

In [30]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Видим, что есть отрицательное значение "-1" и значение "20" вряд ли соответствующее действительности.
Скорее всего в обоих случаях имеет место ошибка ввода - случайный ввод лишнего знака, т.к. количество таких значений мало. Но на всякий случай просмотрим строки с этими значениям

In [31]:
data[data['children'] == 20].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880,21,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья
720,20,855,44,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости
1074,20,3310,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518,получение образования
2510,20,2714,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474,операции с коммерческой недвижимостью
2941,20,2161,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля
3302,20,1808,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,135514,профильное образование
3396,20,1808,56,высшее,0,женат / замужем,0,F,компаньон,0,135514,высшее образование
3671,20,913,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255,на покупку подержанного автомобиля
3697,20,2907,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380,на покупку подержанного автомобиля
3735,20,805,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200,ремонт жилью


Какой-то корреляции с данными из других столбцов не видно. Кстати, видим, теперь уже визуально, что заполнение пропусков в столбцах days_employed и total_income прошло успешно

In [32]:
data[data['children'] == -1].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816,профильное образование
705,-1,902,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882,приобретение автомобиля
742,-1,3174,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268,дополнительное образование
800,-1,1808,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293,дополнительное образование
941,-1,1808,57,Среднее,1,женат / замужем,0,F,пенсионер,0,135514,на покупку своего автомобиля
1363,-1,1195,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550,профильное образование
1929,-1,1461,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121,покупка жилья
2073,-1,2539,42,среднее,1,в разводе,3,F,компаньон,0,162638,покупка жилья
3814,-1,3045,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892,на проведение свадьбы
4201,-1,901,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375,операции со своей недвижимостью


Аналогично: корреляции с данными из других столбцов нет. 

Заменим аномальные значения:
- -1 на 1
- 20 на 2

Думаю именно так ошиблись люди при вводе

In [33]:
data[data['children'] == -1] = data[data['children'] == -1].replace(-1, 1)
data[data['children'] == 20] = data[data['children'] == 20].replace(20, 2)

Проверим успешность замены

In [37]:
data[data['children'] == -1]['children'].count()

0

Значений -1 нет

In [38]:
data[data['children'] == 20]['children'].count()

0

Значний 20 тоже нет. Замена прошла успешно

### Работа со столбцом dob_years

In [16]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Аномальное значение "0". Просмотрим эти строки:

In [17]:
data[data['dob_years'] == 0].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,1808,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
149,0,2664,0,среднее,1,в разводе,3,F,сотрудник,0,70176,операции с жильем
270,3,1872,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166,ремонт жилью
578,0,1808,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1040,0,1158,0,высшее,0,в разводе,3,F,компаньон,0,303994,свой автомобиль
1149,0,934,0,среднее,1,женат / замужем,0,F,компаньон,0,201852,покупка недвижимости
1175,0,1808,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949,получение дополнительного образования
1386,0,5043,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523,сделка с автомобилем
1890,0,1808,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,135514,жилье
1898,0,1808,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400,на покупку автомобиля


Какой-либо корреляции с данными из других столбцов нет.

Посчитаем количество аномалий

In [18]:
data[data['dob_years'] == 0]['dob_years'].count()

101

На весь датафрейм в 21000 с лишним строк, всего 101 такая запись. Кроме того, исходя из постановки задачи выяснить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, думаю, что можно не менять эти значения сейчас.

### Работа со столбцом education и education_id

Выявим уникальные значения в столбце education

In [19]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Видно, что есть неявные дубликаты, отличающиеся регистром. Исправим это и проверим результат:

#### Приведение к нижнему регистру

In [20]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Работа с образованием закончена

### Работа со столбцом family_status и family_status_id

Проверим уникальные значения в столбце

In [21]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Дубликатов нет, но эстетическое удовольствие портит заглавная буква. Исправим это заодно и проверим результат

In [22]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Теперь порядок!

Проверим столбец family_status_id

In [23]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Порядок! id категорий тоже пять и по количеству всё сходится

### Работа со столбцами gender, income_type, debt

Проверим на аномалии столбец gender

In [24]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Интересный пол какой попался. Посмотрим на эту строку

In [25]:
data.loc[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


В контексте предстоящего исследования пол не важен. Просто будем знать что в одной строке пол неизвестен

Проверим столбец income_type

In [26]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Дубликатов не видно.

Проверим столбец debt

In [27]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Аномалий нет.

Посмотрим на общую информацию о получившемся датафреме

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Как видим из общей информации, пропусков нет. Визуально оценим получившийся датафрейм (на этот раз с конца)

In [29]:
data.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,467,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486,заняться образованием
21516,0,914,42,высшее,0,женат / замужем,0,F,компаньон,0,322807,покупка своего жилья
21517,0,404,42,высшее,0,гражданский брак,1,F,компаньон,0,178059,на покупку своего автомобиля
21518,0,1808,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем
21519,1,2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,1808,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля
21524,2,1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля


Читаемость таблицы на высоте.

### Поиск и удаление явных дубликатов

Проверим наличие явных дубликатов

In [30]:
data[data.duplicated() == True]['children'].count()

71

71 дубликат необходимо удалить. Займемся этим и проверим результат

In [31]:
data = data.drop_duplicates().reset_index(drop=True)
data[data.duplicated() == True]['children'].count()

0

Дубликаты удалены

### Итоги предобработки

Были выявлены проблемы в виде отрицательных значений, 
аномально больших значений, пропусков, неявных и явных дубликатов. 

Что было сделано:
- пропуски были заменены нулевыми значениями, чтобы будущая смена типа сработала без ошибок
- Тип **float** был сменен на тип **int**, для улучшения читаемости таблицы
- Отрицательные значения были заменены положительными
- **Аномально большие и нулевые значения** заменены медианами по соответствующему столбцу, т.к. именно такая замена внесет **наименьшие искажения** в результат исследования.
- Выявлены неявные дубликаты в столбцах с категориальными переменными. Записи приведены к единому регистру, что избавило нас от неявных дубликатов
- После такой "приборки" были выявлены и удалены явные дубликаты

При поиске неявных дубликатов в столбцах с небольшим для визуального восприятия количеством уникальных значений, применялся метод **value_counts()**. Результат применения выглядит, как Series, что проще воспринимается. Метод **unique()** применялся для поиска аномалий в столбце с возрастом, т.к. уникальных значений в нем больше и все они числовые - в виде списка вполне читается.

## Оптимизация датафрейма и анализ данных

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим "словари" с категориями образования и семейного положения и проверим результат работы кода, выведя на экран значения созданных для "словарей" переменных

In [32]:
education_dict = data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [33]:
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Порядок. Теперь удалим столбцы с категориями семейного положения и образования и проверим результат

In [34]:
data = data.drop(['family_status', 'education'], axis=1)
data.columns

Index(['children', 'days_employed', 'dob_years', 'education_id',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

Выбранные столбцы удалены

### Категоризация дохода.

Напишем код функции для категоризации дохода и проверим её работу

In [35]:
def income_id(income):
    if income <= 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    else:
        return 'A'

test_income = [200, 35000, 55000, 250000, 1100000]
for income in test_income:
    print(income_id(income))

E
D
C
B
A


Функция работает корректно.

Теперь создадим отдельный столбец с категориями дохода и проверим результат визуально

In [36]:
data['income_id'] = data['total_income'].apply(income_id)
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_id
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,1808,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Категоризация целей кредита.

Проверим уникальные значения в столбце с целями кредита для выявления признаков, по которым будем присваивать категорию

In [37]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

После анализа всех уникальных значений выявил следующий характерные строки, которые позволят присвоить категорию всем значениям столбца с целями кредита:

- "авто"
- "недвиж"
- "жиль"
- "свадьб"
- "образ"

Напишем функцию для присвоения категорий, создадим новый столбец с ними и проверим результат:

In [38]:
def purpose_id(purpose):
    if purpose.find('авто') != -1:
        return 'операции с автомобилем'
    elif purpose.find('недвиж') != -1:
        return 'операции с недвижимостью'
    elif purpose.find('свадьб') != -1:
        return 'проведение свадьбы'
    elif purpose.find('образ') != -1:
        return 'получение образования'
    else:
        return 'операции с недвижимостью'

data['purpose_id'] = data['purpose'].apply(purpose_id)
print(data['purpose_id'].value_counts())

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_id, dtype: int64


Как видим, категории присвоены. На всякий пожарный проверим наличие пропусков в новом столбце с категориями:

In [39]:
data[data['purpose_id'].isna() == True]['purpose_id'].count()

0

Категоризация проведена успешно

### Анализ данных. Ответы на вопросы.

Для ответов на поставленные вопросы будем использовать сводные таблицы по соответствующим столбцам и функцию mean() для нахождения процента невозвращённых кредитов по каждой группе.

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Гипотеза, соответствующая данному вопросу звучит так: чем больше детей, тем больший процент невозврата кредита. Выведем сводную таблицу и проверим гипотезу:

In [40]:
children_pivot = pd.pivot_table(data, index='children', values='debt', aggfunc='mean')
children_pivot

,debt
children,
0,0.075438
1,0.091658
2,0.094925
3,0.081818
4,0.097561
5,0.000000


Как видим значительный рост (почти 2%) невозвратов есть между группой заёмщиков без детей и с одним ребенком. В остальных случаях при увеличении количества детей идёт либо незначительный рост, либо снижение процента невозвратов. Более того нужно учитывать, что объёмы выборки по каждому количеству детей разные. Поэтому нулевой процент невозврата у заёмщиков с 5 детьми не говорит о том, что всем им нужно выдать, да побольше. Просто объём выборки маленький.

**Вывод**: гипотеза не подтвердилась **количество** детей **не влияет** на процент невозврата кредита. 

**Влияет наличие детей:**
- **есть дети - процент невозврата выше;**
- **нет детей - процент невозврата ниже;**

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Гипотеза такая: люди в официальном браке более надежные заёмщики (что логично, нечужие люди друг другу помогают)

Выведем сводную таблицу по семейному положению и невозврату кредита. Также, для удобства интерпретации присоединим столбец с расшифровкой идентификаторов семейного положения.

In [42]:
family_status_pivot = pd.pivot_table(data, index='family_status_id', values='debt', aggfunc=my_mean)
family_status_pivot = family_status_pivot.merge(family_status_dict, on='family_status_id', how='left')
family_status_pivot

,family_status_id,debt,family_status
0,0,7.55%,женат / замужем
1,1,9.35%,гражданский брак
2,2,6.57%,вдовец / вдова
3,3,7.11%,в разводе
4,4,9.75%,не женат / не замужем


По представленным данным выделил бы низкие показания значений "в разводе", "женат / замужем", "вдовец / вдова".

**Вывод:**
гипотеза подтверждена. Более того, судя по этим данным, **более надежны люди, когда-либо состоявшие в официальном браке**, а не только состоящие в таковом на момент подачи заявки. Значит готовность оформить отношения официально характеризует человека, как более надежного.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Гипотеза по этому вопросу: чем выше доход, тем меньше процент невозврата.

Словарь присоединять в данном случае нет необходимости. Закономерность простая: 
- категория "А" - максимальный доход;
- категория "E" - минимальный доход;

In [43]:
pd.pivot_table(data, index='income_id', values='debt', aggfunc=my_mean)

,debt
income_id,
A,8.00%
B,7.06%
C,8.49%
D,6.00%
E,9.09%


Гипотеза явно не подтвердилась, даже с учётом разного объёма выборок. Хотя люди с минимальным доходом не возвращают деньги чаще, чем все остальные. 

**Вывод:** Прямой закономерности процента невозврата и уровня дохода нет, но есть более надёжные категории "D" и "B"

#### Как разные цели кредита влияют на его возврат в срок?

Гипотеза: более дорогие покупки провоцируют больший процент невозврата

In [44]:
pd.pivot_table(data, index='purpose_id', values='debt', aggfunc=my_mean)

,debt
purpose_id,
операции с автомобилем,9.36%
операции с недвижимостью,7.23%
получение образования,9.22%
проведение свадьбы,8.00%


Недвижимость в данной выборке наиболее дорогостоящая, а процент невозврата самый низкий. Также низкий процент имеет категория "проведение свадьбы".

**Вывод:** гипотеза опровергнута, скорее нужно говорить о более надёжных категориях целей, которые подразумевают ответственность нескольких людей. Недвижимость в одиночку тяжеловато купить, да и чаще всего это кредит с поручительством, а свадьба - это вступление в официальный брак, что, как мы выяснили выше характеризует заёмщиков, как более надёжных.

## Общий вывод:

Анализируя **проблемы**, которые были выявлены в ходе предобработки исходных данных, думаю, что большинство из них **вводятся вручную** человеком. Отсюда разный регистр, и опечатки. Для предотвращения подобного в будущем, предложил бы максимально **сократить ручной ввод с клавиатуры** и пользоваться, например, **выпадающими списками**, в полях анкеты, в которых может быть небольшое количество уникальных значений.

Отдельно стоит сказать о столбце со стажем работы. Так как он посчитан в днях, а ни один здравомыслящий банковский сотрудник не будет спрашивать/вводить стаж в днях, предполагаю, что существует какая-то **функция**, которая принимает дату начала трудовой деятельности, затем с помощью текущей даты вычисляет стаж и переводит его в дни. **Ошибки в работе этой функции** и являются причиной некорректных данных в исходном файле. Для повышения качества данных для анализа вдальнейшем, очень нужно исправить эти ошибки

По итогам же анализа данных получился **портрет идеального заёмщика** по четырём критериям:

- **Нет детей.** Процент невозврата - 7.54 %
- **Вдовец/вдова.** Процент невозврата - 6.57 %
- **Цель кредита: операции с недвижимостью.** Процент невозврата - 7.23 %
- **Доход: от 30001 до 50000.** Процент невозврата - 6.00 %

**Худший заёмщик** имеет следующие характеристики:

- **4 ребенка.** Процент невозврата - 9.76 %
- **Не женат/не замужем.** Процент невозврата - 9.75 %
- **Цель кредита: операции с автомобилем.** Процент невозврата - 9.36 %
- **Доход: менее 30 000.** Процент невозврата - 9.09 %